In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df= pd.read_csv('spam_ham_dataset.csv')
df.sort_values(by=['label_num'], inplace=True, ascending=False)
df = df[:2*len(df[df["label_num"]==1])]
df =df.reset_index(drop=True)
df['label_num'].mean()

0.5

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
def filter_words(text):
    words = text.split()
    stop_words = set(stopwords.words("english"))
    non_alphabet_pattern = re.compile(r'[^a-zA-Z]')
    filtered_words = [word for word in words if word not in stop_words]
    filtered_words = [re.sub(non_alphabet_pattern, '', word) for word in filtered_words]
    filtered_words = list(filter(None, filtered_words))
    return filtered_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
words_dict=[]
words_ =[]
for i in range(len(df)):
    f =filter_words(df['text'][i])
    words_dict.extend(f)
    words_.append(f)
df['words'] = words_
words_dict = list(set(words_dict))
words_ =None

In [ ]:
words_dict = pd.DataFrame(words_dict)

In [ ]:
print(words_dict)

                0
0             sus
1          goethe
2      savonarola
3           furze
4       blutwurst
...           ...
41766  impactions
41767      snares
41768      missin
41769    ndglhffn
41770      lineno

[41771 rows x 1 columns]


In [ ]:
words_dict['id']=words_dict.index
dict_ = words_dict.to_dict('list')
words_dict =None

In [ ]:
dict_

{0: ['sus',
  'goethe',
  'savonarola',
  'furze',
  'blutwurst',
  'forecasts',
  'cnusc',
  'summarizes',
  'znrlt',
  'eeghenstraat',
  'zsv',
  'backyardi',
  'ayk',
  'mollyl',
  'lesser',
  'regulated',
  'definately',
  'aggressive',
  'bulky',
  'revisions',
  'valet',
  'alps',
  'meditation',
  'witch',
  'inadvertently',
  'penultimate',
  'currently',
  'networking',
  'exposition',
  'repathed',
  'eac',
  'louisa',
  'revoiutionary',
  'hid',
  'crocus',
  'glib',
  'pareto',
  'suresh',
  'relay',
  'ya',
  'afloat',
  'secu',
  'xinjvevk',
  'lbhawh',
  'ppjqvfcsq',
  'nephew',
  'claiming',
  'prepared',
  'delray',
  'htmabout',
  'sightseeing',
  'badbadbadgurl',
  'enemy',
  'npvjkps',
  'trysts',
  'vaous',
  'erratic',
  'finish',
  'behold',
  'petrofina',
  'bmi',
  'angeies',
  'usda',
  'fastow',
  'imywuca',
  'thyrotoxic',
  'rqhc',
  'zigqykxl',
  'sladana',
  'devastate',
  'eros',
  'widowhood',
  'midwives',
  'rkhjrvyuj',
  'poagrjcep',
  'agm',
  'mzqp

In [ ]:
np.random.seed(0)
df = df.sample(frac=1).reset_index(drop=True)
train_data = df[:int(len(df)*0.8)].reset_index(drop=True)
test_data = df[int(len(df)*0.8):].reset_index(drop=True)

In [ ]:
train_data_0 = train_data[train_data['label_num']==0].reset_index(drop=True)
train_data_1 = train_data[train_data['label_num']==1].reset_index(drop=True)

train_features_0 = {}
for i in dict_['id']:
    train_features_0[i] =[]
for i in range(len(train_data_0)):
    words_ =np.array(train_data_0['words'][i])
    for word in set(words_):
        if word in dict_[0]:
            freq = (words_==word).sum()
            id_ = dict_['id'][dict_[0].index(word)]
            train_features_0[id_].append(freq)


train_features_1 = {}
for i in dict_['id']:
    train_features_1[i] =[]
for i in range(len(train_data_1)):
    words_ =np.array(train_data_1['words'][i])
    for word in set(words_):
        if word in dict_[0]:
            freq = (words_==word).sum()
            id_ = dict_['id'][dict_[0].index(word)]
            train_features_1[id_].append(freq)

In [ ]:
mean_var_0 = {}
for i in dict_['id']:
    if train_features_0[i]==[]:
        mean_var_0[i] =[0.0001,0.0001]
        continue
    mean = np.mean(train_features_0[i])
    var = np.var(train_features_0[i])
    mean_var_0[i] =[mean+0.0001,var+0.0001]

mean_var_1 = {}
for i in dict_['id']:
    if train_features_1[i]==[]:
        mean_var_1[i] =[0.0001,0.0001]
        continue
    mean = np.mean(train_features_1[i])
    var = np.var(train_features_1[i])
    mean_var_1[i] =[mean+0.0001,var+0.0001]


In [ ]:
def gaussian_prob_0(x, id_):
    mean = mean_var_0[id_][0]
    var = mean_var_0[id_][1]
    coeff = 1.0 / np.sqrt(2.0 * np.pi * var)
    exponent = np.exp(-(np.power(x - mean, 2) / (2 * var)))
    return coeff * exponent + 0.00001

def gaussian_prob_1(x, id_):
    mean = mean_var_1[id_][0]
    var = mean_var_1[id_][1]
    coeff = 1.0 / np.sqrt(2.0 * np.pi * var)
    exponent = np.exp(-(np.power(x - mean, 2) / (2 * var)))
    return coeff * exponent + 0.00001

In [ ]:
def pois_prob_0(x, id_):
    mean = mean_var_0[id_][0]
    return np.exp(-mean)*np.power(mean,x)/np.math.factorial(x)

def pois_prob_1(x, id_):
    mean = mean_var_1[id_][0]
    return np.exp(-mean)*np.power(mean,x)/np.math.factorial(x)

In [ ]:
def bernoulli_prob_0(x, id_):
    mean = mean_var_0[id_][0]
    return np.power(mean,x)*np.power(1-mean,1-x)

def bernoulli_prob_1(x, id_):
    mean = mean_var_1[id_][0]
    return np.power(mean,x)*np.power(1-mean,1-x)

In [ ]:
def multino_prob_0(x, id_):
    mean = mean_var_0[id_][0]
    return np.power(mean,x)

def multino_prob_1(x, id_):
    mean = mean_var_1[id_][0]
    return np.power(mean,x)

In [ ]:
p_0 = len(train_data_0)/len(train_data)
p_1 = len(train_data_1)/len(train_data)
print(p_0,p_1)

0.5012510425354462 0.4987489574645538


In [ ]:

pred =[]
for i in range(len(test_data)):
    prob_0 =1
    prob_1 =1
    for word in test_data['words'][i]:
        id_ = dict_[0].index(word)
        freq = (np.array(test_data['words'][i])==word).sum()
        prob_0 *= gaussian_prob_0(freq, id_)
        prob_1 *= gaussian_prob_1(freq, id_)
    prob_0 = (p_0*prob_0)/((p_0*prob_0)+(p_1*prob_1))
    prob_1 = (p_1*prob_1)/((p_0*prob_0)+(p_1*prob_1))

    if prob_0>prob_1:
        pred.append(0)
    else :
        pred.append(1)


<ipython-input-18-fec927a7338f>:10: RuntimeWarning: invalid value encountered in double_scalars
  prob_0 = (p_0*prob_0)/((p_0*prob_0)+(p_1*prob_1))


In [ ]:
print("Accuracy: ", (np.array(pred)==np.array(test_data['label_num'])).sum()/len(test_data))

Accuracy:  0.86


In [ ]:
import scipy.special

# ...

def pois_prob_0(x, id_):
    mean = mean_var_0[id_][0]
    return np.exp(-mean) * np.power(mean, x) / scipy.special.factorial(x)

def pois_prob_1(x, id_):
    mean = mean_var_1[id_][0]
    return np.exp(-mean) * np.power(mean, x) / scipy.special.factorial(x)

# ...


In [ ]:
print("Accuracy: ", (np.array(pred)==np.array(test_data['label_num'])).sum()/len(test_data))

Accuracy:  0.855


In [ ]:
pred =[]
for i in range(len(test_data)):
    prob_0 =1
    prob_1 =1
    for word in test_data['words'][i]:
        id_ = dict_[0].index(word)
        freq = (np.array(test_data['words'][i])==word).sum()
        prob_0 *= bernoulli_prob_0(freq, id_)
        prob_1 *= bernoulli_prob_1(freq, id_)
    prob_0 = (p_0*prob_0)/((p_0*prob_0)+(p_1*prob_1))
    prob_1 = (p_1*prob_1)/((p_0*prob_0)+(p_1*prob_1))

    if prob_0>prob_1:
        pred.append(0)
    else :
        pred.append(1)

<ipython-input-22-12ad02611d26>:9: RuntimeWarning: overflow encountered in double_scalars
  prob_1 *= bernoulli_prob_1(freq, id_)
<ipython-input-22-12ad02611d26>:11: RuntimeWarning: invalid value encountered in double_scalars
  prob_1 = (p_1*prob_1)/((p_0*prob_0)+(p_1*prob_1))
<ipython-input-22-12ad02611d26>:8: RuntimeWarning: overflow encountered in double_scalars
  prob_0 *= bernoulli_prob_0(freq, id_)
<ipython-input-22-12ad02611d26>:10: RuntimeWarning: invalid value encountered in double_scalars
  prob_0 = (p_0*prob_0)/((p_0*prob_0)+(p_1*prob_1))
<ipython-input-15-ad3e37379199>:7: RuntimeWarning: overflow encountered in power
  return np.power(mean,x)*np.power(1-mean,1-x)


In [ ]:
print("Accuracy: ", (np.array(pred)==np.array(test_data['label_num'])).sum()/len(test_data))

Accuracy:  0.7966666666666666


In [ ]:
pred =[]
for i in range(len(test_data)):
    prob_0 =1
    prob_1 =1
    for word in test_data['words'][i]:
        id_ = dict_[0].index(word)
        freq = (np.array(test_data['words'][i])==word).sum()
        prob_0 *= multino_prob_0(freq, id_)
        prob_1 *= multino_prob_1(freq, id_)
    prob_0 = (p_0*prob_0)/((p_0*prob_0)+(p_1*prob_1))
    prob_1 = (p_1*prob_1)/((p_0*prob_0)+(p_1*prob_1))

    if prob_0>prob_1:
        pred.append(0)
    else :
        pred.append(1)

<ipython-input-24-c45d5df064e2>:8: RuntimeWarning: overflow encountered in double_scalars
  prob_0 *= multino_prob_0(freq, id_)
<ipython-input-24-c45d5df064e2>:10: RuntimeWarning: invalid value encountered in double_scalars
  prob_0 = (p_0*prob_0)/((p_0*prob_0)+(p_1*prob_1))
<ipython-input-24-c45d5df064e2>:9: RuntimeWarning: overflow encountered in double_scalars
  prob_1 *= multino_prob_1(freq, id_)
<ipython-input-24-c45d5df064e2>:11: RuntimeWarning: invalid value encountered in double_scalars
  prob_1 = (p_1*prob_1)/((p_0*prob_0)+(p_1*prob_1))


In [ ]:
print("Accuracy: ", (np.array(pred)==np.array(test_data['label_num'])).sum()/len(test_data))

Accuracy:  0.855
